### 배민상회 기준 카테고리를 분류하기위해 Naive Bayes 활용
> 민형기 강사님 pdf자료 29번, 34번 참고

- 여러 케이스로 시도함.. 뭐가 더 알맞는지 판단할 수준이 아니였음..
- 아래의 다른 clf는 LGBMClassifier, XGBClassifier 사용함
    - case 1. : countvector -> tfidftransformer -> multinomialNB / 다른 clf
    - case 2. : tfidfvector -> tfidftransformer -> multinomialNB / 다른 clf
    - case 3. : nltk.naivebayesclassifier
    
|case/acc | multinomialNB | LGBMClassifier | XGBClassifier| naivebayesclassifier |
|:----:|:---------------:|:--------------:|:------------:|:------------:|
|case 1 | 0.65 | 0.48 | 0.57 | - |
|case 2 | 0.79 | - | 0.87 | - |
|case 3 | - | - | - | 0.82 |


> ### 좋은 시도였다.. 깨달은점.
1. **사전 학습시키기 위한 데이터는 충분해야한다..(최소 500개 이상)**
2. 이진 분류일 경우 naivebayes, 그 이상일 경우 multinomialNB
3. 판단 기준은 accuracy 보다는 confusion_matrix를 봐야한다\
*(이를 수치로 나타낼 수 있을까? 더 낫다 라는걸 비교할 수있을까...? roc그래프를 그려야하나?)*

In [251]:
# %load baemin_mart_items.py

import pandas as pd
import requests 
import time

categories = {
    100015 : "냉동상품",
    100024 : "냉장식품",
    100028 : "유제품",
    100032 : "식용유/설탕/소금",
    100038 : "장류/조미료",
    100043 : "소스/양념/육수",
    100050 : "밀가루/라면/믹스류",
    100054 : "편의식/가공기타",
    100061 : "반찬류",
    100067 : "야채/채소",
    100077 : "과일",
    100081 : "쌀/잡곡/견과",
    100085 : "수산/건어물",
    100096 : "축산/계란",
}

def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])
        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

# global gagong_df -> 파이썬 파일을 실행 했을 때 gagong_df가 나왔으면 좋겠음.. 
baemin_mart_items_df = pd.concat(dfs, ignore_index=True)
baemin_mart_items_df

# baemin_mart_items.to_csv("baemin_mart_items.csv", index=False, encoding="utf-8-sig")

print("="*100)
print("It's saved as baemin_mart_items.csv file.")
print("="*100)


냉동상품
0 1 2 3 4 5 6 7 8 9 10 11 
냉장식품
0 1 2 3 4 5 6 7 8 9 10 11 
유제품
0 1 2 3 4 5 6 7 8 9 10 11 
식용유/설탕/소금
0 1 2 3 4 5 6 7 8 9 10 11 
장류/조미료
0 1 2 3 4 5 6 7 8 9 10 11 
소스/양념/육수
0 1 2 3 4 5 6 7 8 9 10 11 
밀가루/라면/믹스류
0 1 2 3 4 5 6 7 8 9 10 11 
편의식/가공기타
0 1 2 3 4 5 6 7 8 9 10 11 
반찬류
0 1 2 3 4 5 6 7 8 9 10 11 
야채/채소
0 1 2 3 4 5 6 7 8 9 10 11 
과일
0 1 2 3 4 5 6 7 8 9 10 11 
쌀/잡곡/견과
0 1 2 3 4 5 6 7 8 9 10 11 
수산/건어물
0 1 2 3 4 5 6 7 8 9 10 11 
축산/계란
0 1 2 3 4 5 6 7 8 9 10 11 
It's saved as baemin_mart_items.csv file.


In [254]:
bm_df1 = baemin_mart_items_df[['title', 'category']]
bm_df1.tail()

,title,category
4429,[든든] 도드람 양념목심 500g 5개,축산/계란
4430,[든든] 도드람 양념왕구이 630g 5개,축산/계란
4431,[든든] 도드람 오소리감투 200g 5개,축산/계란
4432,[든든] 도드람 통갈비양념구이 800g 5개,축산/계란
4433,[든든] 도드람 훈제바베큐족발 500g,축산/계란


In [255]:
bm_df1['category'].value_counts()

편의식/가공기타      1011
냉동상품           762
소스/양념/육수       564
수산/건어물         489
축산/계란          486
장류/조미료         276
야채/채소          229
밀가루/라면/믹스류     156
반찬류            114
쌀/잡곡/견과         99
식용유/설탕/소금       92
유제품             78
냉장식품            61
과일              17
Name: category, dtype: int64

In [256]:
# %load baemin_mart_delivery_goods.py

import pandas as pd
import requests 
import time

# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])
        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

# global gagong_df -> 파이썬 파일을 실행 했을 때 gagong_df가 나왔으면 좋겠음.. 
baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*100)
print("It's saved as baemin_mart_delivery_goods.csv file.")
print("="*100)


포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 
It's saved as baemin_mart_delivery_goods.csv file.


In [257]:
bm_df2 = baemin_mart_delivery_goods_df[['title', 'category']]
bm_df2.tail()

,title,category
2390,[든든]리오 락스 2L,가게위생용품
2391,[든든]스카치브라이트 수세미(다목적용_청색) 2 개,가게위생용품
2392,[든든]크린업 부직포행주(삼색_38*38cm) 3 개,가게위생용품
2393,[든든]크린업 스폰지수세미(15*9*3cm) 2 개,가게위생용품
2394,[든든]크린업 은사수세미(18*14cm) 2 개,가게위생용품


In [312]:
bm_df = pd.concat([bm_df1, bm_df2], ignore_index=True)
bm_df.tail()

,title,category
6824,[든든]리오 락스 2L,가게위생용품
6825,[든든]스카치브라이트 수세미(다목적용_청색) 2 개,가게위생용품
6826,[든든]크린업 부직포행주(삼색_38*38cm) 3 개,가게위생용품
6827,[든든]크린업 스폰지수세미(15*9*3cm) 2 개,가게위생용품
6828,[든든]크린업 은사수세미(18*14cm) 2 개,가게위생용품


### case 1. countvector -> tfidftransformer -> multinomialNB / 다른 clf

In [40]:
# import string 
# from nltk.corpus import stopwords

In [65]:
# bm_df['title'][0]

'[든든] 닭목살튀김 마일드     '

In [61]:
# bm_df['title'][len(bm_df['title'])-1]

'[든든]크린업 은사수세미( * )  '

In [62]:
# 특수문자 제거 ( !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ )

In [78]:
# s = [char for char in bm_df['title'][0] if char not in string.punctuation]
# s = ''.join(s)
# s.split()

['닭목살튀김', '마일드']

In [79]:
# (숫자)g/kg/개 , 든든 제거 

In [80]:
# s = re.sub(r"\d+\s?\w*", " ", s)
# s = re.sub(r"든든", " ", s)
# s.split()

['닭목살튀김', '마일드']

In [44]:
# 불용어 제거 
# 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 
# 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있습니다. 
# 이러한 단어들을 불용어(stopword)라고 함. -> 한글은 따로 설정해서 하는 듯.
# 혹은 tfidf에서 많이 등장하는 단어 max_df로 걸러내는 듯. 

In [338]:
def clean_txt(text):
    s = [char for char in text if char not in string.punctuation]
    s = ''.join(s)
    s = re.sub(r"\d+\s?\w*", " ", s)
    s = re.sub(r"든든", " ", s)

#     nopunc = s.split()
    return s

In [339]:
bm_df['title'] = bm_df['title'].apply(clean_txt)
bm_df['title'].tail()

6824                 리오 락스  
6825     스카치브라이트 수세미다목적용청색  
6826          크린업 부직포행주삼색   
6827           크린업 스폰지수세미   
6828            크린업 은사수세미   
Name: title, dtype: object

In [340]:
from sklearn.model_selection import train_test_split

t_train, t_test, c_train, c_test = train_test_split(bm_df['title'], 
                                                    bm_df['category'], test_size=0.2,
                                                   random_state=13)

In [262]:
from sklearn.feature_extraction.text import (CountVectorizer, 
                                             TfidfTransformer, TfidfVectorizer)
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

### case 1-1. CountVectorizer->TfidfTransformer->MultinomialNB

In [392]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

In [393]:
pipeline.fit(t_train, c_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [394]:
predictions = pipeline.predict(t_test)

In [395]:
from sklearn.metrics import accuracy_score
accuracy_score(c_test,predictions)

0.650805270863836

In [396]:
from sklearn.metrics import classification_report
print(classification_report(c_test,predictions))

              precision    recall  f1-score   support

      가게위생용품       1.00      0.19      0.31        59
          과일       0.00      0.00      0.00         3
        냉동상품       0.63      0.59      0.61       146
        냉장식품       0.00      0.00      0.00        15
  밀가루/라면/믹스류       1.00      0.05      0.09        22
         반찬류       0.00      0.00      0.00        23
    소스/양념/육수       0.75      0.66      0.70       122
      수산/건어물       1.00      0.30      0.46       105
   식용유/설탕/소금       0.00      0.00      0.00        20
     쌀/잡곡/견과       0.00      0.00      0.00        17
       야채/채소       0.83      0.14      0.23        37
         유제품       0.00      0.00      0.00        13
      장류/조미료       0.90      0.39      0.54        49
       축산/계란       0.91      0.67      0.77        90
    편의식/가공기타       0.72      0.79      0.75       228
        포장용품       0.57      1.00      0.72       417

    accuracy                           0.65      1366
   macro avg       0.52   

c:\users\82106\anaconda3\envs\fc14th\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [397]:
from sklearn.metrics import confusion_matrix
cnf = confusion_matrix(c_test,predictions)
print(cnf)

[[ 11   0   3   0   0   0   0   0   0   0   0   0   0   0   6  39]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   1   1]
 [  0   0  86   0   0   0   2   0   0   0   0   0   0   3  18  37]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   3  10]
 [  0   0   1   0   1   0   7   0   0   0   0   0   1   0   3   9]
 [  0   0   1   0   0   0   0   0   0   0   0   0   0   0   7  15]
 [  0   0  13   0   0   0  80   0   0   0   0   0   0   0   5  24]
 [  0   0   1   0   0   0   0  31   0   0   0   0   0   3  13  57]
 [  0   0   6   0   0   0   6   0   0   0   0   0   1   0   4   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1  16]
 [  0   0   2   0   0   0   0   0   0   0   5   0   0   0   0  30]
 [  0   0   3   0   0   0   0   0   0   0   0   0   0   0   3   7]
 [  0   0   9   0   0   0   6   0   0   0   0   0  19   0   1  14]
 [  0   0   4   0   0   0   0   0   0   0   0   0   0  60   5  21]
 [  0   0   6   0   0   0   6   0   0   0   0   0   0   0 179 

In [398]:
# 무슨.. 삼겹살이... 포장용품이니.. ㅠㅠ

In [399]:
pipeline.predict(['삼겹살'])

array(['포장용품'], dtype='<U10')

### case 1-2. CountVectorizer->TfidfTransformer->LGBMClassifier

In [406]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LGBMClassifier())
])

In [407]:
pipeline.fit(t_train, c_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LGBMClassifier())])

In [408]:
predictions = pipeline.predict(t_test)

In [409]:
accuracy_score(c_test,predictions)

0.48316251830161056

In [410]:
print(classification_report(c_test,predictions))

              precision    recall  f1-score   support

      가게위생용품       0.12      0.03      0.05        59
          과일       0.00      0.00      0.00         3
        냉동상품       0.31      0.21      0.25       146
        냉장식품       0.00      0.00      0.00        15
  밀가루/라면/믹스류       0.00      0.00      0.00        22
         반찬류       0.00      0.00      0.00        23
    소스/양념/육수       0.53      0.33      0.40       122
      수산/건어물       0.45      0.14      0.22       105
   식용유/설탕/소금       0.42      0.25      0.31        20
     쌀/잡곡/견과       0.00      0.00      0.00        17
       야채/채소       0.50      0.11      0.18        37
         유제품       0.00      0.00      0.00        13
      장류/조미료       0.56      0.20      0.30        49
       축산/계란       0.96      0.57      0.71        90
    편의식/가공기타       0.75      0.38      0.51       228
        포장용품       0.45      1.00      0.62       417

    accuracy                           0.48      1366
   macro avg       0.32   

c:\users\82106\anaconda3\envs\fc14th\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [412]:
pipeline.predict(['진라면'])

array(['포장용품'], dtype=object)

### case 1-3. CountVectorizer->TfidfTransformer->XGBClassifier

In [420]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier())
])

In [421]:
pipeline.fit(t_train, c_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=No

In [422]:
predictions = pipeline.predict(t_test)

In [423]:
accuracy_score(c_test,predictions)

0.5739385065885798

In [424]:
print(classification_report(c_test,predictions))

              precision    recall  f1-score   support

      가게위생용품       0.25      0.20      0.22        59
          과일       0.10      0.33      0.15         3
        냉동상품       0.51      0.44      0.47       146
        냉장식품       0.50      0.07      0.12        15
  밀가루/라면/믹스류       0.69      0.41      0.51        22
         반찬류       0.88      0.30      0.45        23
    소스/양념/육수       0.73      0.43      0.54       122
      수산/건어물       0.61      0.24      0.34       105
   식용유/설탕/소금       0.54      0.35      0.42        20
     쌀/잡곡/견과       1.00      0.18      0.30        17
       야채/채소       0.00      0.00      0.00        37
         유제품       0.00      0.00      0.00        13
      장류/조미료       0.61      0.41      0.49        49
       축산/계란       0.94      0.69      0.79        90
    편의식/가공기타       0.83      0.46      0.60       228
        포장용품       0.52      1.00      0.68       417

    accuracy                           0.57      1366
   macro avg       0.54   

In [425]:
print(confusion_matrix(c_test,predictions))

[[ 12   0   5   0   0   0   0   0   0   0   0   0   0   0   0  42]
 [  0   1   0   0   0   0   0   0   0   0   0   0   0   0   1   1]
 [  6   1  64   1   0   1   3   0   0   0   0   0   5   2   6  57]
 [  2   0   0   1   0   0   0   0   0   0   1   0   0   0   1  10]
 [  0   0   1   0   9   0   2   0   1   0   0   0   4   0   0   5]
 [  2   0   3   0   0   7   0   0   0   0   0   0   0   0   2   9]
 [ 10   1  20   0   1   0  52   0   3   0   0   0   2   0   2  31]
 [  3   0   5   0   0   0   0  25   0   0   1   1   0   1   3  66]
 [  1   0   5   0   1   0   4   0   7   0   0   0   0   0   1   1]
 [  0   0   0   0   0   0   0   0   0   3   0   0   0   0   1  13]
 [  2   4   0   0   0   0   0   0   0   0   0   0   0   0   0  31]
 [  0   1   2   0   0   0   0   0   0   0   0   0   0   0   1   9]
 [  3   0   3   0   0   0   7   0   2   0   0   0  20   0   0  14]
 [  0   0   1   0   0   0   0   0   0   0   0   0   0  62   3  24]
 [  5   2  17   0   2   0   3  16   0   0   0   1   2   1 106 

In [426]:
pipeline.predict(['진라면'])

array(['포장용품'], dtype=object)

In [427]:
pipeline.predict(['[국내산] 유명산지 햇사과 5~8입/봉'])

array(['축산/계란'], dtype=object)

> ### 분류기를 MultinomialNB 했을 때 accuracy 0.65로 가장 높음

### case 2. tfidfvector -> tfidftransformer -> multinomialNB / 다른 clf

In [428]:
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    token = okt.morphs(text)
    return token

In [429]:
okt_tokenizer('진경바보')

['진경', '바보']

In [430]:
tfidf_v = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 2), max_df=0.9)
tfidf_v.fit(t_train)
tfidf_train = tfidf_v.transform(t_train)

c:\users\82106\anaconda3\envs\fc14th\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


### case 1-1. TfidfVectorizer->TfidfTransformer->MultinomialNB

In [431]:
nb_clf = MultinomialNB()
nb_clf.fit(tfidf_train, c_train)

MultinomialNB()

In [432]:
from sklearn.metrics import accuracy_score

tfidf_test = tfidf_v.transform(t_test)
predict = nb_clf.predict(tfidf_test)

accuracy_score(c_test, predict)

0.7935578330893118

### case 1-2. TfidfVectorizer->TfidfTransformer->XGBClassifier

In [433]:
xgb_clf = XGBClassifier()
xgb_clf.fit(tfidf_train, c_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [434]:
predict = xgb_clf.predict(tfidf_test)

accuracy_score(c_test, predict)

0.8711566617862372

In [435]:
print(confusion_matrix(c_test, predict))

[[ 51   0   2   2   0   0   0   0   0   0   0   0   0   0   4   0]
 [  0   1   0   0   0   0   0   0   0   0   1   0   0   0   0   1]
 [  2   0 120   1   0   0   0   0   0   0   1   1   2   4  14   1]
 [  1   0   4   6   0   0   0   0   0   0   0   0   0   0   4   0]
 [  0   0   1   0  16   0   0   0   0   0   0   0   1   0   3   1]
 [  0   0   0   0   0  14   1   0   0   0   0   0   1   1   5   1]
 [  0   0   4   0   0   0 105   0   0   0   0   0   1   1  11   0]
 [  0   1   1   0   0   0   1  88   0   0   1   0   1   1  10   1]
 [  1   0   1   0   0   0   0   0  15   0   0   0   1   0   2   0]
 [  0   0   1   0   0   0   0   0   0  13   0   0   0   0   3   0]
 [  0   0   0   0   0   3   0   0   0   0  33   0   0   0   1   0]
 [  0   0   0   0   0   0   0   1   0   0   0   6   0   0   6   0]
 [  0   0   1   0   0   0   2   1   2   1   0   0  37   0   5   0]
 [  0   0   1   0   0   0   2   0   0   0   5   0   1  71  10   0]
 [  4   0  13   0   1   0   0   5   0   1   0   1   2   1 198 

In [436]:
xgb_clf.predict(tfidf_v.transform(['진라면']))

array(['편의식/가공기타'], dtype=object)

In [437]:
xgb_clf.predict(tfidf_v.transform(['[국내산] 유명산지 햇사과 5~8입/봉']))

array(['야채/채소'], dtype=object)

In [438]:
xgb_clf.predict(tfidf_v.transform(['[목우촌] 주부9단 김밥햄 340g(170g*2개입)']))

array(['냉동상품'], dtype=object)

> ### 분류기를 XGBClassifier 했을 때 accuracy 0.87로 가장 높음. 실제 예측 결과는 ... 애매함..

### case 3. nltk.naivebayesclf

In [359]:
train = [ _ for _ in zip(t_train, c_train)]
train[:5]

[(' 이츠웰 홀 양송이캔  ', '편의식/가공기타'),
 ('  동경단무지 슬라이스  ', '반찬류'),
 ('  도야지 옛날맛편육  ', '편의식/가공기타'),
 ('앞치마일회용맘편히자알', '가게위생용품'),
 ('고강도 도시락용기 ', '포장용품')]

In [360]:
train_mor = [(okt.morphs(x[0]), x[1]) for x in train]
train_mor[:5]

[(['이츠웰', '홀', '양송이', '캔'], '편의식/가공기타'),
 (['동경', '단무지', '슬', '라이스'], '반찬류'),
 (['도야지', '옛날', '맛', '편육'], '편의식/가공기타'),
 (['앞치마', '일회용', '맘', '편히자', '알'], '가게위생용품'),
 (['고', '강도', '도시락', '용기'], '포장용품')]

In [361]:
all_words = set([ t for x in train_mor for t in x[0]])
# all_words

In [362]:
def term_exists(doc):
    return {word : (word in set(doc)) for word in all_words}

In [439]:
train_data = [(term_exists(d), c) for d, c in train_mor]

In [447]:
# train_data[0][0]

In [364]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(train_data)
classifier.show_most_informative_features()

Most Informative Features
                      용기 = True             포장용품 : 편의식/가공 =    300.4 : 1.0
                       용 = True            축산/계란 : 포장용품   =    172.3 : 1.0
                       돈 = True            축산/계란 : 편의식/가공 =    160.0 : 1.0
                      소스 = True           소스/양념/ : 축산/계란  =    120.4 : 1.0
                       산 = True            야채/채소 : 편의식/가공 =    114.1 : 1.0
                       면 = True           밀가루/라면 : 냉동상품   =    102.1 : 1.0
                       내 = True            야채/채소 : 포장용품   =    101.9 : 1.0
                      백설 = True           식용유/설탕 : 냉동상품   =     98.6 : 1.0
                       식 = True           밀가루/라면 : 포장용품   =     87.4 : 1.0
                       왕 = True           밀가루/라면 : 포장용품   =     76.7 : 1.0


In [365]:
test_mor = [okt.morphs(x) for x in t_test]

In [366]:
test_data = [term_exists(x) for x in test_mor]

In [367]:
test_cate = [classifier.classify(x) for x in test_data]

In [368]:
accuracy_score(c_test, test_cate)

0.821376281112738

In [370]:
cnf = confusion_matrix(c_test, test_cate)
print(cnf)

[[ 42   0   8   0   0   0   0   0   0   0   0   0   0   0   0   9]
 [  0   0   1   0   0   0   0   1   0   0   0   0   0   0   1   0]
 [  0   0 132   0   0   0   3   0   0   0   0   0   2   4   3   2]
 [  0   0  11   0   0   0   1   1   0   0   0   0   0   0   1   1]
 [  0   0   2   0   7   0   6   0   0   0   0   0   1   0   3   3]
 [  0   0   2   0   0   6   2   4   0   0   1   0   0   1   4   3]
 [  0   0   2   0   0   0 113   0   0   0   0   0   0   0   3   4]
 [  0   0   3   0   0   0   1  91   0   0   0   0   0   2   2   6]
 [  0   0  11   0   0   0   5   1   0   0   0   0   0   0   1   2]
 [  0   0   5   0   0   0   0   0   0   4   0   0   2   0   2   4]
 [  0   0   1   0   0   0   0   0   0   0  35   0   0   0   1   0]
 [  0   0   4   0   0   0   1   0   0   0   0   0   0   0   8   0]
 [  0   0   0   0   0   0  11   0   0   0   0   0  34   0   1   3]
 [  0   0   8   0   0   0   1   0   0   0   0   0   0  71   5   5]
 [  0   0  18   0   1   0   6   8   0   0   0   0   1   0 176 

In [372]:
classifier.classify(term_exists(okt.morphs('[목우촌] 주부9단 김밥햄 340g(170g*2개입)')))

'편의식/가공기타'

In [373]:
classifier.classify(term_exists(okt.morphs('[국내산] 유명산지 햇사과 5~8입/봉')))

'축산/계란'

> naivebayes 였음에도 불구하고 accuracy가 높아 테스트를 해보았으나.. 이또한 애매했다..